In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
plt.rcParams['figure.figsize'] = [8, 4.5]

In [ ]:
data_root = Path("benchmarks/results/whisper_benchmark")

In [ ]:
datasets = {p.stem.removeprefix("NVIDIA"): pd.read_csv(p) for p in data_root.glob("*.csv")}

In [ ]:
def prepare_data(datasets):
    for dataset in datasets.values():
        dataset.rename(columns={"time":"Latency [s]", "wer": "WER", "energy": "Energy [kJ]", "model_size": "Model Size"}, inplace=True)
        dataset["1-WER"] = 1 - dataset["WER"]
        dataset["Energy [kJ]"] = dataset["Energy [kJ]"] / 1_000
        dataset["WERDP [s]"] = dataset["Latency [s]"] * dataset["WER"]
        dataset["WERE [kJ]"] = dataset["WER"] * dataset["Energy [kJ]"]
        dataset["WEREDP [kJs]"] = dataset["WER"] * dataset["Energy [kJ]"] * dataset["Latency [s]"]
        dataset["Relative Latency"] = dataset["Latency [s]"] / 10 * 60

In [ ]:
prepare_data(datasets)

In [ ]:
def plot_vars(datasets: dict, x,y,title, annotate=True, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    markers_n_spacings = iter([("o", 0.02), ("^",0.04), ("x", 0.04)])
    for gpu, data in datasets.items():
        marker, spacing = next(markers_n_spacings)
        ax.plot(data[x] ,data[y] , marker, label=gpu)
        ax.set_title("Whisper " + title)
        ax.set_xlabel(x)
        ax.set_ylabel(y)
        #plt.ylim(bottom=0)
        ax.legend()
        if annotate:
            max_x = data[x].max()
            min_x = data[x].min()

            for index, size in data["Model Size"].items():
                ax.annotate(size, (data[x][index] + spacing * (max_x - min_x) , data[y][index]), va="center")
    return ax

In [ ]:
def plot_latency_vs_wer(datasets: dict, ax=None):
    return plot_vars(datasets, "Latency [s]", "1-WER", "Latency vs. 1-WER",ax=ax)

def plot_energy_vs_wer(datasets: dict, ax=None):
    return plot_vars(datasets, "Energy [kJ]", "1-WER", "Energy vs. 1-WER",ax=ax)

def plot_latency_vs_energy(datasets: dict, ax=None):
    return plot_vars(datasets, "Latency [s]", "Energy [kJ]", "Latency vs. Energy",ax=ax)

In [ ]:
def plot_werdp_vs_model_size(datasets, ax =None):
    return plot_vars(datasets, "Model Size", "WERDP [s]", "Word Error Rate Delay Product vs. Model Size", annotate=False, ax=ax)

def plot_werdp_vs_energy(datasets, ax =None):
    return plot_vars(datasets, "Energy [kJ]", "WERDP [s]", "Word Error Rate Delay Product vs. Model Size", annotate=True, ax=ax)

def plot_were_vs_model_size(datasets, ax =None):
    return plot_vars(datasets, "Model Size", "WERE [kJ]", "Word Error Rate Energy vs. Model Size", annotate=False, ax=ax)

def plot_weredp_vs_model_size(datasets, ax =None):
    return plot_vars(datasets, "Model Size", "WEREDP [kJs]", "Word Error Rate Energy Delay Product vs. Model Size", annotate=False, ax=ax)

In [ ]:
plt.style.use("seaborn-v0_8-colorblind")
plt.style.use("seaborn-v0_8-talk")
fig, ax = plt.subplots(ncols=3,figsize=(24,4.5))
plot_latency_vs_wer(datasets, ax[0])
plot_energy_vs_wer(datasets,ax[1])
plot_latency_vs_energy(datasets, ax[2])
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(ncols=2,nrows=2,figsize=(16,9))
plot_werdp_vs_model_size(datasets, ax[0,0])
plot_werdp_vs_energy(datasets, ax[0,1])
plot_were_vs_model_size(datasets, ax[1,0])
plot_weredp_vs_model_size(datasets, ax[1,1])
fig.tight_layout()